# Events Example

## Installation

For readability we've written a simple python interface for the Signal API under `signal_api/signalAPI.py` from the root of this repository.

Make sure you've installed the dependencies for these notebooks:

```bash
cd signal-api-tools # make sure you're in the root of this project
pip install -e . -r requirements.txt
```


In [1]:
import signal_api.signalAPI as signalAPI
import pandas as pd
from itertools import groupby, islice
import os
from tqdm import tqdm
import json

pd.set_option('display.max_colwidth', 0)
pd.options.display.html.use_mathjax = False

## Credentials

The notebook assumes that API credentials are stored as environment variables.

In [2]:
client_id = os.environ['SIGNAL_API_ID']
client_secret = os.environ['SIGNAL_API_SECRET']
signal_api = signalAPI.Connection(client_id=client_id, client_secret=client_secret)

## Searching for events using Entities, Topics

Let's search for events about the entity `Long Covid` and the topics `Healthcare`, `Healthcare Crisis` or  `Transformation of Healthcare`.

In [3]:
entities = {
    'Long Covid': '1021d2f6-e57a-308c-b232-d6c6f2c2588e',
}

# uncomment line below to use all signal entities
# entities = {}

topics = {
    'Healthcare': 'ec838d2b-49db-457a-b42f-2889fb86b2eb',
    'Healthcare Crisis': '0d6c8ffc-d2fd-47bc-b304-fff55e76cd19',
    'Transformation of Healthcare': 'c3f7b9d6-9420-46fb-abb2-490382212189'
}

# uncomment line below to use all signal topics
# topics = {} 

# uncomment to search events for specific labels
# A label can contain a maximum of 6 words
labels = {
    #'analysis of workers compensation claims'
}

# uncomment to search for events mentioned by specific sources
sources = {
#     'Yahoo News UK': '008a1423-3f16-4de1-8b7e-b0ee2b793655',
#     'Washington Post': '38663047-4bae-42e8-9474-bd6201f1f492'
}

You can search for signal concepts and their IDs as follows. For more details see the getting started notebook.

In [4]:
list(islice(signal_api.entities({'name': 'Covid'}), 3))

[{'id': 'e2e3b3e9-e552-3377-9705-e48c8be05227',
  'type': 'disease',
  'name': 'COVID 19'},
 {'id': '1021d2f6-e57a-308c-b232-d6c6f2c2588e',
  'type': 'disease',
  'name': 'Long COVID'},
 {'id': '532095ac-5836-301a-a313-f6b86b926d75',
  'type': 'product',
  'name': 'Covid 19 Vaccine'}]

Use the last year as a date range

In [5]:
end_date = pd.to_datetime('today').to_period('D') - 1
start_date = end_date - 365
start_date, end_date

(Period('2022-04-05', 'D'), Period('2023-04-05', 'D'))

## Query the Events API

Below the craft a query as input to the events API

In [6]:
query = {
      "where": {
          "date": {
              "gte": str(start_date),
              "lte": str(end_date)
      },
    },
    "size": 1000
}

if entities:
  query['where']["entities"] = {
    "id": {
        # include events about any of the entities
        "any": list(entities.values())

        # note you can seach for events about all of the entities too
        # "all": list(entities.values())
    }
  }

if topics: 
  query['where']["topics"] = {
    "id": {
        # include events about any of the topics
        "any": list(topics.values())

        # note you can seach for events about all of the topics too
        # "all": list(topics.values())
    }
  }

if labels:
  query['where']["labels"] = {
    # include events about any of the labels
    "any": list(labels)

    # note you can seach for events about all of the labels too
    # "all": list(labels.values())
    
  }

if sources: 
  query['where']["sources"] = {
    "id": {
        # include events about any of the sources
        "any": list(sources.values())

        # note you can seach for events about all of the sources too
        # "all": list(sources.values())
    }
  }

print('Using events query:')
print(json.dumps(query, indent=4))

Using events query:
{
    "where": {
        "date": {
            "gte": "2022-04-05",
            "lte": "2023-04-05"
        },
        "entities": {
            "id": {
                "any": [
                    "1021d2f6-e57a-308c-b232-d6c6f2c2588e"
                ]
            }
        },
        "topics": {
            "id": {
                "any": [
                    "ec838d2b-49db-457a-b42f-2889fb86b2eb",
                    "0d6c8ffc-d2fd-47bc-b304-fff55e76cd19",
                    "c3f7b9d6-9420-46fb-abb2-490382212189"
                ]
            }
        }
    },
    "size": 1000
}


Now lets call the events API using or query, taking the first 10 results

In [7]:
response = signal_api.events(query)
# remove results that come from a small number of sources
response = (event for event in response if event['source-count-global'] >= (event['story-count-global'] / 5))
# take the first 5 events
events = list(islice(response, 5))

We can see the result from the events API below.

Note that each event contains a list of story IDs. 
These are the documents that constitute the event in order of relevance. In the next section we will use these `story-ids` to fetch document level metadata such as headlines for each event from the search endpoint.

In [8]:
pd.json_normalize(events)

,labels,date,document-count-entitlements,hash,story-count-global,story-ids,document-count-global,entities,topics,source-count-global
0,"[mild, long covid, symptoms, long covid symptoms, infection, year, study]",2023-01-12,566,02e5-8f8b9366-932bc9df-b1aa3367-f6f9b7c7,192,"[541cf63c-f2f7-48ee-bb75-51cf32565dd1, f9727514-fae4-48d1-b081-bdcee837bae4, 3cd76e0c-232f-4ffb-aa07-25ddb8325123, aab6c7bd-8478-407b-914d-43f0091fb0c8, 345c7d57-9032-41ad-b025-044e2a5e5ff9, 2d44b59c-ef25-4da4-a915-40abac2415e2, 025c08cb-01a6-4530-83c2-51de420bba86, 057434f2-6ab7-4d54-9e15-fbe985b73eb9, 7e35cb0a-26d4-4bda-a33b-2db1d6f7d6c0, abb7a840-9361-43a4-a4e7-416f174f0d10, d3420112-1db3-472d-9d4c-43a6587f8b84, 7af32d99-50a4-42ed-b513-f292ca74a3f1, 3a7dd038-4619-4a8b-8ad9-7a1eb1efc913, dac2e327-c881-4e9f-93c0-179279be51d5, 17cdc916-2d70-45d2-aba8-8f93b2c5ca28, 9226b093-6bc8-4f8d-8f2f-427bcd0cf2bf, d00271e9-06ff-4ea5-9d7a-0c26b42429fe, 3666881c-b3b6-40e3-83b3-e045dbd3254e, 2d4dd863-c616-4715-b33f-7db56664dda4, 137de471-44f3-453a-b6a1-8a37700acdef, e2a00f00-ee8b-4582-bc64-5e563a8537ca, 85716889-4f65-48ba-93d1-8fc8323d0851, 87f3deef-ac42-4b3e-bbd7-5a6144f259ee, ea0d1c9c-8185-40ef-8e32-2725e477e222, e40e7ba8-34cb-466a-91d0-922cecb45fd9, 163b9d01-2dbf-42de-abc3-7216ce6da7e8, ae9cecb6-891c-49c9-8ab4-d41fb4aab787, 50b1501a-21ac-4808-9568-609e09373d26, fb0b0120-d591-48b7-8b26-790f8d979610, 0b5b172b-34b8-4424-967b-96d5b706653c, c67bd667-ac97-4eb2-897b-0d3e9babda29, 8fd34cc8-9d49-4410-be54-e5adf9a1b65e, 0872d765-826e-42b9-a3cb-73669fb4d38b, 69105891-d32c-47ef-b34a-1f389bebde16, d245803e-6f27-4af6-b776-66c8c4228d6c, 542ee41c-f77b-4fa1-b8d8-1f51be7ec06a, c07377e3-e991-4f48-ab31-b9f72272be82, 4ac5953e-cbed-4fcf-901f-db2b4179e7e5, cae0cbb0-d59d-4fd7-8c74-5ca3db65e484, 1c14a1bc-653d-42ba-ae90-c447eae3078f, df3b281d-ca8a-4822-84fe-65861a2c1156, 88c0e502-7f35-4bf9-b480-862ef97484c5, 1500207c-1356-4b94-9f05-5d7c01108772, fb5289e1-167b-4886-ba05-b741247a6560, b6b3b862-1f1e-4628-a09f-1d82f0f61e72, e4f9dbd1-437d-4723-9d35-5999e35b9f33, a3e3f67a-f1be-48ed-9ff2-8ffe852d0788, c00a29c2-80f1-428a-b75b-89d7729daa7c, 509847de-b392-4fa2-855d-146030a19105, 3a19902c-1e3c-48a5-ad38-a24147021758, ca35802a-a46c-486d-9da3-b9dfe39e88bb, 8ac792a9-d9d9-4d74-81aa-b990fd2f32d9, 2273d8e7-c0e6-470b-a4c3-46afefe12b63, 689e5c8e-68be-4730-8c78-08da87bcfd88, 8f0cfd5e-071f-406e-ba50-99f71aacf60f, 3491629c-452b-4bf3-9329-72126b31360c, d0a6241a-ba98-4acf-8837-07bd634f82e2, 8ad7c1ec-3489-4979-bcd7-32a984f9a5b4, e3ad15d9-ca98-4215-8b63-b518a57128c4, f6420035-e5a8-4a47-88e1-0b61a44a7c48, e29fbe46-2be4-4a72-a239-346ce38f4b2a, 4a5eb42c-e2bd-48f4-844f-6016cb5ee9d8, 57c01792-18ce-478a-9cc4-2a843174d4a5, 2f9b0004-b76d-44a5-8d4b-497124cc6956, e52b0480-fe54-4bc8-a69b-5dbba6705ee0, c2ef56eb-edd9-42f1-ba08-caed1e149ec1, 3b85c483-5b5f-4e63-9432-a89983c79bbb, 408c536f-ad6e-4340-a041-8c4facdc8fb9, e7656111-3683-443d-bcb0-654ed29769dd, a9b040e1-15ef-4a65-b93f-3bfbb440a1d4, 8f470001-0d7e-48a7-9405-6c13e69c799e, f5fffe67-9de3-45ce-b0b2-317e398e3d83, 45a0edee-b16c-4866-bb78-52acea6d0ffa, da4b53f0-c274-46e0-bc64-ef4c9ea9f2b8, cd5f3f53-4387-4889-bad1-3b0fdc5b287b, 525feb8d-f76f-48f5-89fe-6ce581bb63cb, b0272563-de5b-41ae-8a55-305dd2db13c7, 32923fda-a51a-40da-a64d-0cb1c3962d0b, 3f333d2a-fa57-405e-ba0b-5c9e72e70cc5, d351ff68-43bf-4ded-aa86-51b0c5b450a7, d7688c32-2a9d-4cf6-b49f-a282bc7535bb, 61bc2903-d412-4086-bf38-15a11cc1f2d9, bd20adfa-6c75-400f-9da2-5ede0d3b71c9, 20fbcb42-7383-4105-8a6a-532c04bace05, 80cde8f7-e675-49ce-95a0-312fdd590dff, 3abbe2b4-613d-496e-abc2-7202dc7704a1, 8446c6b0-24eb-4c37-b67b-dce9c4cb8730, 808a2755-50da-4882-8141-988d80b3aa2e, e5e9b044-68e7-4ac6-926b-e568bff7ffe2, d91bbf84-f8d7-4603-811f-d368b11cb937, 46c09577-f539-4e1c-b179-b26749eb2083, fbcd6f2c-6aa0-4e0b-9bd8-a2bff0157f11, 4f48b703-9645-47ce-95e3-64d769cc56bc, 96c87d28-72fd-49b1-8e05-819b0c3a1e96, 9dcd7785-c43c-465c-8673-7551c2f11c01, fd8e109d-327c-4ad8-a75b-73074684ac3d, db2af368-adcc-4725-83ac-67a2f0e19a22, a181e0cb-ceec-47ce-a176-8d7f563af8db, f76

## Fetching Headlines for Events

For each event let's take the list of `story-ids` and transform them into a list of headlines using the function below.

In [9]:
def search_by_story_id(story_ids: list, entity_ids: list, topic_ids: list, source_ids: list = None,  n=3):
    """
    return one document for each story-id in "story_ids" 
    return the results in the same order as story_ids
    """
    # record the ordering of the story_ids
    rank = {story_id: rnk for rnk, story_id in enumerate(story_ids)}

    # query the seach endpoint
    params = {
        'where': {
            'story-id': {
                'any': list(story_ids)
            },
        },
        'size': 500
    }

    if entity_ids:
        params['where']['entities'] = {
            'id': {'any': entity_ids},
            'salient-only': True
        }
        
    if topic_ids:
        params['where']['topics'] = {
            'id': {'any': topic_ids}
        }
    
    if source_ids:
        params['where']['source'] = {
            'id': {'any': source_ids}
        }

    response = (item['document'] for item in signal_api.search(params))

    # put the results in the same order as the story IDs
    ordered = sorted(response, key=lambda document: rank[document['story-id']])

    # deduplicate the results
    grouped = islice(groupby(ordered, lambda x: x['story-id']), n)
    return [next(group) for _, group in grouped]

### Get documents from the Search API

In [10]:
for event in tqdm(events, desc='Calling Search Endpoint'):
    # arg = {
    #     'story_ids': event['story-ids'],
    #     'entity_ids': list(entities.values()), 
    #     'topic_ids': list(topics.values())
    # }

    # if sources:
    #     arg['source_ids'] = list(sources.values())
    event['documents'] = search_by_story_id(
        story_ids=event['story-ids'], 
        entity_ids=list(entities.values()) if entities else None, 
        topic_ids=list(topics.values()) if topics else None,
        source_ids=list(sources.values()) if sources else None
    )
    

Calling Search Endpoint: 100%|██████████| 5/5 [00:21<00:00,  4.25s/it]


## Display Events with Headlines

Now let's view the events with their respective headlines

In [11]:
events_df = pd.DataFrame([
    {
        'hash': event['hash'],
        'date': event['date'],
        'story-count-global': event['story-count-global'],
        'source-count-global': event['source-count-global'],
        'entities': [entity['name'] for entity in event['entities']],
        'topics': [topic['name'] for topic in event['topics']][:3], # just use the first 3 topics 
        'labels': event['labels'],
        'headlines': [doc['title'] for doc in event['documents']],
    }
    for event in events
])
events_df

,hash,date,story-count-global,source-count-global,entities,topics,labels,headlines
0,02e5-8f8b9366-932bc9df-b1aa3367-f6f9b7c7,2023-01-12,192,479,"[COVID 19, Long COVID]","[Health Technology, Health Systems, Healthcare]","[mild, long covid, symptoms, long covid symptoms, infection, year, study]","[Study found that symptoms of long COVID disappear within a year, in mild cases, Long COVID symptoms go away within a year, in mild cases, Majority of long covid symptoms in mild cases may disappear after one year | coronavirus]"
1,023b-1b1bd948-b7b7c757-99ef6a79-e4c1f6a3,2022-07-26,189,420,"[COVID 19, Long COVID]","[Health Technology, Health Systems, Medical Research]","[long covid, symptoms, hair loss, dysfunction]","[News Hair Loss and Low Sex Drive Added to List of Long Covid Symptoms Research from the University of Birmingham has revealed that symptoms of long COVID are broader than expected and include hair loss and sexual dysfunction., Hair loss and sexual dysfunction are new long COVID symptoms, Hair loss, low sex drive among long Covid symptoms: Study]"
2,02c8-53c99b2e-8f21c57f-9d403e16-fea17fc3,2022-12-14,173,815,"[COVID 19, Long COVID, Centers for Disease Control and Prevention]","[Healthcare Crisis, Health Systems, Health Technology]","[long covid, deaths, cdc, 500, centers for disease control and prevention]","[CDC: Long COVID behind more than 3,500 deaths, CDC: Long COVID behind more than 3,500 deaths, CDC: Long COVID behind more than 3,500 deaths]"
3,023a-1b1bd148-97b36756-99ef6e65-e6e1f7cb,2022-07-25,103,604,"[COVID 19, Long COVID]","[Health Technology, Health Systems, Healthcare]","[hair loss, symptoms, long covid symptoms, long covid, study]","[Hair and libido loss join fatigue and brain fog among wider list of Long COVID symptoms, Hair loss and decreased sex drive on broader list of long Covid symptoms - study, Hair and libido loss join fatigue and brain fog among wider list of Long Covid symptoms]"
4,026c-9bcaf816-133dc9ee-9856e927-b5f0f7d6,2022-09-13,98,603,"[COVID 19, Long COVID, Europe]","[Health Technology, Health Systems, Healthcare]","[long covid, 17 million people in europe, symptoms, years of the pandemic, 17 million, world health organization]","[17 million people in Europe got long Covid in the first two years of the pandemic, WHO: 17 million Europeans have long covid symptoms, WHO says 17 million people in Europe have long Covid symptoms]"


## Get event by hash

By default only limited information is returned when searching for events. To get all the metadata about an event use the event hash.

In [12]:
# You can retrieve an event by hash to get additional metadata about the event such as source ids 
# and additional story ids

event_hash = events_df.iloc[0]['hash']
event = signal_api.get_event(event_hash)

In [13]:
pd.json_normalize(event)

,labels,date,document-count-entitlements,hash,story-count-global,story-ids,document-count-global,entities,source-ids,topics
0,"[mild, long covid, symptoms, long covid symptoms, infection, year, study]",2023-01-12,566,02e5-8f8b9366-932bc9df-b1aa3367-f6f9b7c7,192,"[541cf63c-f2f7-48ee-bb75-51cf32565dd1, f9727514-fae4-48d1-b081-bdcee837bae4, 3cd76e0c-232f-4ffb-aa07-25ddb8325123, aab6c7bd-8478-407b-914d-43f0091fb0c8, 345c7d57-9032-41ad-b025-044e2a5e5ff9, 2d44b59c-ef25-4da4-a915-40abac2415e2, 025c08cb-01a6-4530-83c2-51de420bba86, 057434f2-6ab7-4d54-9e15-fbe985b73eb9, 7e35cb0a-26d4-4bda-a33b-2db1d6f7d6c0, abb7a840-9361-43a4-a4e7-416f174f0d10, d3420112-1db3-472d-9d4c-43a6587f8b84, 7af32d99-50a4-42ed-b513-f292ca74a3f1, 3a7dd038-4619-4a8b-8ad9-7a1eb1efc913, dac2e327-c881-4e9f-93c0-179279be51d5, 17cdc916-2d70-45d2-aba8-8f93b2c5ca28, 9226b093-6bc8-4f8d-8f2f-427bcd0cf2bf, d00271e9-06ff-4ea5-9d7a-0c26b42429fe, 3666881c-b3b6-40e3-83b3-e045dbd3254e, 2d4dd863-c616-4715-b33f-7db56664dda4, 137de471-44f3-453a-b6a1-8a37700acdef, e2a00f00-ee8b-4582-bc64-5e563a8537ca, 85716889-4f65-48ba-93d1-8fc8323d0851, 87f3deef-ac42-4b3e-bbd7-5a6144f259ee, ea0d1c9c-8185-40ef-8e32-2725e477e222, e40e7ba8-34cb-466a-91d0-922cecb45fd9, 163b9d01-2dbf-42de-abc3-7216ce6da7e8, ae9cecb6-891c-49c9-8ab4-d41fb4aab787, 50b1501a-21ac-4808-9568-609e09373d26, fb0b0120-d591-48b7-8b26-790f8d979610, 0b5b172b-34b8-4424-967b-96d5b706653c, c67bd667-ac97-4eb2-897b-0d3e9babda29, 8fd34cc8-9d49-4410-be54-e5adf9a1b65e, 0872d765-826e-42b9-a3cb-73669fb4d38b, 69105891-d32c-47ef-b34a-1f389bebde16, d245803e-6f27-4af6-b776-66c8c4228d6c, 542ee41c-f77b-4fa1-b8d8-1f51be7ec06a, c07377e3-e991-4f48-ab31-b9f72272be82, 4ac5953e-cbed-4fcf-901f-db2b4179e7e5, cae0cbb0-d59d-4fd7-8c74-5ca3db65e484, 1c14a1bc-653d-42ba-ae90-c447eae3078f, df3b281d-ca8a-4822-84fe-65861a2c1156, 88c0e502-7f35-4bf9-b480-862ef97484c5, 1500207c-1356-4b94-9f05-5d7c01108772, fb5289e1-167b-4886-ba05-b741247a6560, b6b3b862-1f1e-4628-a09f-1d82f0f61e72, e4f9dbd1-437d-4723-9d35-5999e35b9f33, a3e3f67a-f1be-48ed-9ff2-8ffe852d0788, c00a29c2-80f1-428a-b75b-89d7729daa7c, 509847de-b392-4fa2-855d-146030a19105, 3a19902c-1e3c-48a5-ad38-a24147021758, ca35802a-a46c-486d-9da3-b9dfe39e88bb, 8ac792a9-d9d9-4d74-81aa-b990fd2f32d9, 2273d8e7-c0e6-470b-a4c3-46afefe12b63, 689e5c8e-68be-4730-8c78-08da87bcfd88, 8f0cfd5e-071f-406e-ba50-99f71aacf60f, 3491629c-452b-4bf3-9329-72126b31360c, d0a6241a-ba98-4acf-8837-07bd634f82e2, 8ad7c1ec-3489-4979-bcd7-32a984f9a5b4, e3ad15d9-ca98-4215-8b63-b518a57128c4, f6420035-e5a8-4a47-88e1-0b61a44a7c48, e29fbe46-2be4-4a72-a239-346ce38f4b2a, 4a5eb42c-e2bd-48f4-844f-6016cb5ee9d8, 57c01792-18ce-478a-9cc4-2a843174d4a5, 2f9b0004-b76d-44a5-8d4b-497124cc6956, e52b0480-fe54-4bc8-a69b-5dbba6705ee0, c2ef56eb-edd9-42f1-ba08-caed1e149ec1, 3b85c483-5b5f-4e63-9432-a89983c79bbb, 408c536f-ad6e-4340-a041-8c4facdc8fb9, e7656111-3683-443d-bcb0-654ed29769dd, a9b040e1-15ef-4a65-b93f-3bfbb440a1d4, 8f470001-0d7e-48a7-9405-6c13e69c799e, f5fffe67-9de3-45ce-b0b2-317e398e3d83, 45a0edee-b16c-4866-bb78-52acea6d0ffa, da4b53f0-c274-46e0-bc64-ef4c9ea9f2b8, cd5f3f53-4387-4889-bad1-3b0fdc5b287b, 525feb8d-f76f-48f5-89fe-6ce581bb63cb, b0272563-de5b-41ae-8a55-305dd2db13c7, 32923fda-a51a-40da-a64d-0cb1c3962d0b, 3f333d2a-fa57-405e-ba0b-5c9e72e70cc5, d351ff68-43bf-4ded-aa86-51b0c5b450a7, d7688c32-2a9d-4cf6-b49f-a282bc7535bb, 61bc2903-d412-4086-bf38-15a11cc1f2d9, bd20adfa-6c75-400f-9da2-5ede0d3b71c9, 20fbcb42-7383-4105-8a6a-532c04bace05, 80cde8f7-e675-49ce-95a0-312fdd590dff, 3abbe2b4-613d-496e-abc2-7202dc7704a1, 8446c6b0-24eb-4c37-b67b-dce9c4cb8730, 808a2755-50da-4882-8141-988d80b3aa2e, e5e9b044-68e7-4ac6-926b-e568bff7ffe2, d91bbf84-f8d7-4603-811f-d368b11cb937, 46c09577-f539-4e1c-b179-b26749eb2083, fbcd6f2c-6aa0-4e0b-9bd8-a2bff0157f11, 4f48b703-9645-47ce-95e3-64d769cc56bc, 96c87d28-72fd-49b1-8e05-819b0c3a1e96, 9dcd7785-c43c-465c-8673-7551c2f11c01, fd8e109d-327c-4ad8-a75b-73074684ac3d, db2af368-adcc-4725-83ac-67a2f0e19a22, a181e0cb-ceec-47ce-a176-8d7f563af8db, f766983b-cfb

### Show the first 10 sources that mentioned this event

In [14]:

[signal_api.get_source(source_id)['source']['name'] for source_id in tqdm(event['source-ids'][:10])]

100%|██████████| 10/10 [00:01<00:00,  5.54it/s]


['Advertiser (Australia)',
 'Yahoo News UK',
 'Headlines Today',
 'Isle of Wight Radio',
 'Head Topics',
 'Yahoo! Noticias',
 'T13.cl',
 'Radio Essex',
 'Knowledia News (IL)',
 'The Windsor Star']